# E09 Spiking neural networks

In this week's homework, we are going to use the code from the tutorial and look at the effect of changes in its parameters in a more systematic way.

In [ ]:
# Import Brian and switch off its "code generation" facility
from brian2 import *
prefs.codegen.target = 'numpy'
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# This code is slightly adapted from the tutorial (lower number of neurons and shorter run time,
# and it additionally returns spike counts), see notes in the tutorial for more explanation about the code.

def run_sim(input_rate, g, N_E=1000):
    start_scope()
    # network parameters
    gamma = 0.25
    N_I = int(gamma * N_E)
    N = N_E + N_I
    epsilon = 0.1
    C_E = C_ext = int(epsilon * N_E)
    C_I = int(epsilon * N_I)

    # neuron parameters
    tau = 20 * ms; theta = 20 * mV; V_r = 10 * mV; tau_rp = 2 * ms

    # synapse parameters
    J = 0.1 * mV; D = 1.5 * ms

    # external stimulus
    nu_thr = theta / (J * C_E * tau)

    neurons = NeuronGroup(N,"dv/dt = -v/tau : volt (unless refractory)",
                          threshold="v > theta", reset="v = V_r",
                          refractory=tau_rp, method="exact")

    exc_neurons = neurons[:N_E]
    inh_neurons = neurons[N_E:]
    exc_synapses = Synapses(exc_neurons, neurons, on_pre="v += J", delay=D)
    exc_synapses.connect(i='k for k in sample(N_E, size=C_E)')
    inhib_synapses = Synapses(inh_neurons, neurons, on_pre="v += -g*J", delay=D)
    inhib_synapses.connect(i='k for k in sample(N_I, size=C_I)')

    nu_ext = input_rate * nu_thr
    external_poisson_input = PoissonInput(target=neurons, target_var="v",
                                          N=C_ext, rate=nu_ext, weight=J)

    rate_monitor = PopulationRateMonitor(neurons)
    exc_monitor = SpikeMonitor(exc_neurons)
    inh_monitor = SpikeMonitor(inh_neurons[:50])  # record 50 inhibitory neurons
    state_monitor = StateMonitor(exc_neurons, 'v', record=0)  # record membrane potential of one neuron
    run(0.5*second)
    
    return {'rate_t': rate_monitor.t, 'rate': rate_monitor.rate,
            'exc_spikes_t': exc_monitor.t, 'exc_spikes_i': exc_monitor.i, 'exc_spikes_count': exc_monitor.count, 
            'inh_spikes_t': inh_monitor.t, 'inh_spikes_i': inh_monitor.i, 'inh_spikes_count': inh_monitor.count,
            'state_t': state_monitor.t, 'state_v': state_monitor.v[0]}

In our model, each of the neurons in the network receives input via a `PoissonInput`. This models the summed effect of many neurons providing excitatory input to the cell, each spiking randomly according to a Poisson process. This is more efficient than actually simulating each of the external input cells and their synapses individually. 
The `input_rate` parameter that we provide to the `run_sim` function determines the *firing rate* of these external input neurons (as a value relative to the rate that is necessary to have an average input current that drives the neuron to its threshold), i.e. higher values means that each neuron in the network gets stronger external input.

## Question: how does the network activity depend on the input rate and the strength of the inhibition?

To answer this question, we will first simulate the network several times, with different values for the external input rate `input_rate` and for the strength of the inhibitory synapses `g`. For each parameter combination below, store the *mean firing* rate of the excitatory neurons in the `firing_rate` matrix, and the *standard deviation of the firing rates* across excitatory neurons in the `firing_rate_stddev` matrix.

**Hints**:
  * The `results` dictionary returned by `run_sim`, contains an entry `exc_spikes_count` with an array containing the total number of spikes for each neuron.
  * Each simulation runs for 0.5s

In [ ]:
# Parameters that we are varying
g_values = [0, 2, 4]
input_rate_values = np.linspace(0, 5, 6)  # 6 values equally spaced between 0 and 5

# Empty matrices of the appropriate size
firing_rate = np.empty((len(input_rate_values), len(g_values)))
firing_rate_stddev = np.empty((len(input_rate_values), len(g_values)))

print('Running')
for input_rate_index, input_rate_value in enumerate(input_rate_values):
    for g_index, g_value in enumerate(g_values):
        results = run_sim(input_rate_value, g_value)
        # Store mean firing rate: 
        # TODO
        # Store standard deviation of firing rates
        # TODO
        print('.', end='')  # Print a dot to show progress
print('\nFinished')

We will now plot the mean firing rate and its standard deviation over neurons as a function of the external input rate, separately for the different strengths of inhibition.

**Hints**:
  * Use `plt.errorbar` to plot lines (for the mean firing rate) with error bars (for the standard deviation). The syntax is `plt.errorbar(x_values, y_values, error_values)`
  * If you add `label=f'g = {g_value}'` to the `plt.errorbar` call, the legend shows the strength of the inhibition in the label
  * Note that the error bars will most likely be very small and therefore almost invisible. For making sure your plotting code is correct, you can multiply the standard deviations by 10 for the plot – but make sure to remove this multiplication in the final version! 

In [ ]:
for g_index, g_value in enumerate(g_values):
    # TODO: Use plt.errorbar to plot the firing rate for the current value of g over the input_rate, and add
    # error bars for the standard deviation of the firing rate across cells
    # plt.errorbar(...)
    plt.legend()
    plt.ylabel('mean firing rate (spikes/second)')
    plt.xlabel('input strength');

## Question: identify the dynamic regime

Simulate network with the following given values for the `input_rate` and `g`, and plot the results using the provided `plot_results` function (you can use the `title` argument in `plot_results` to give them the corresponding letter as titles). **Use a bigger network with N_E=10000.** The plots will show a raster plot on top, and the population rate on the bottom.

### Values
**A** - `input_rate=2`, `g=3`

**B** - `input_rate=0.9`, `g=4.5`

**C** = `input_rate=4`, `g=6`

**Hint**:

Be aware that these simulations can take a few minutes to finish.

In [ ]:
def plot_results(results, title=None):
    fig, axs = plt.subplots(2, 1, sharex=True, gridspec_kw={"height_ratios": [3, 1]})
    axs[0].plot(results['exc_spikes_t'][results['exc_spikes_i']<50]/ms,
                results['exc_spikes_i'][results['exc_spikes_i']<50], '|')
    axs[0].set(xlim=(300, 500), xticks=[])
    axs[1].plot(results['rate_t']/ms, results['rate']/Hz)
    axs[1].set(xlim=(300, 500), xlabel='t (ms)', ylabel='rate (1/second)')
    if title:
        fig.suptitle(title)

In [ ]:
#TODO: Simulate regime A and plot the results with plot_results

In [ ]:
#TODO: Simulate regime B and plot the results with plot_results

In [ ]:
#TODO: Simulate regime C and plot the results with plot_results

How do the following descriptions map to the above results (i.e. which letter corresponds to which description)?

- **regular + fast oscillations**
- **irregular + slow oscillations**
- **regular + synchronous**

**Hints**:
- regular/irregular describes the regularity of individual cells
- synchronous/asynchronous describes the regularity across cells
- oscillations refers to the activity of the whole population